In [1]:
import pandas as pd
import numpy as np
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv')#, usecols=['lat', 'long', 'id'])
kampala_devices.head()

,name,location,id,lat,long,population_density,altitude,road_distance,residential_road_distance,terrain_90,division
0,aq_79,Ntinda,930434,0.360209,32.610756,4637.08,1216.25,17.46,17.46,21,Nakawa
1,aq_29,Bugolobi,718028,0.307500,32.620600,5576.65,1143.54,303.33,303.33,31,Nakawa
2,aq_69,Nantabulirwa,912224,0.346460,32.703280,530.79,1182.83,26.76,26.76,21,NaN
3,aq_71,Kireka,930426,0.365500,32.646800,1299.25,1188.21,33.39,33.39,31,NaN
4,aq_72,Luwafu,930427,0.268900,32.588000,5924.30,1171.50,27.70,27.70,31,Makindye


In [3]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [4]:
final_df = pd.DataFrame()
# device_ids = []
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number', 'site_name']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
#     if len(device_df)!=0:
#         device_ids.append(device_id)
    processed_df = utils.preprocessing(device_df)
#     print(f'{device_id}: {len(processed_df)}')
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number,site_name
0,453031.0,0.356989,32.613888,10.5477,930434,"Ntinda, Nakawa"
1,453032.0,0.356989,32.613888,16.4250,930434,"Ntinda, Nakawa"
2,453033.0,0.356989,32.613888,17.7239,930434,"Ntinda, Nakawa"
3,453034.0,0.356989,32.613888,16.1533,930434,"Ntinda, Nakawa"
4,453035.0,0.356989,32.613888,18.0123,930434,"Ntinda, Nakawa"


In [5]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
site_names = final_df['site_name'].unique()
len(latitudes), len(longitudes), len(device_ids), len(site_names)

(35, 35, 34, 34)

In [19]:
final_df = final_df.drop(['device_number', 'site_name'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### delete from here

In [40]:
used_kampala_devices = {}
for lat, location in list(zip(list(kampala_devices.lat), list(kampala_devices.location))):
    if lat in final_df.latitude.unique():
        print(f'{lat} for {location}: yes')
        used_kampala_devices[location] = lat
    else:
        print(f'{lat} for {location}: no')

0.360209 for Ntinda: no
0.3075 for Bugolobi: no
0.34646 for Nantabulirwa: no
0.3655 for Kireka: no
0.2689 for Luwafu: no
0.341674 for Banda: no
0.325346 for Mbuya II: no
0.322108 for Nakasero II: no
0.373078 for Kiwatule: no
0.381576 for Kyaliwajjala: no
0.30778 for Butabika: no
0.363 for Nansana West: no
0.3759 for Nansana East: no
0.3101 for Busega: no
0.3875 for Kyanja: no
0.2182 for Busabala: no
0.269993 for Bunamwaya: no
0.3564 for Kyebando: no
0.299 for Lukuli: no
0.3517 for Bukoto I: no
0.29875 for Kansanga: no
0.344 for Kasubi: no
0.3412 for Naguru II: no
0.289 for Makindye I: no
0.3323 for Makerere University: no
0.295314 for Lubaga: no
0.354825 for Bweyogerere: no
0.314 for Civic Centre: no
0.059604 for Kiwafu: no
0.347014 for Kireka: no
0.235668 for Seguku: no
0.2336 for Seguku(2): no
0.2836 for Lukuli(2): no
0.391478 for Najjera: no
0.32232 for Nakasero II(2): no
0.286595 for Kyengera: no
0.307489 for Kisugu: no


In [28]:
device_indices = final_df[final_df.latitude==latitudes[idx]].index
len(device_indices)

1122

In [29]:
test_df = final_df.loc[device_indices]
len(test_df)

1122

In [30]:
train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
len(train_df)

46990

In [31]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((46990, 3), (46990,), (1122, 3), (1122,))

In [32]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [33]:
rmse = sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse, mape

(13.724796577766055, 0.17220297410941773)

#### end here

In [7]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1, 1)
    
    rf = RandomForestRegressor(random_state=42)#, n_estimators=1000, criterion='poisson') #, max_depth=10, criterion='poisson')
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape
    
#     return mean.numpy(), var.numpy(), Xtest, Ytest, round(rmse, 2)

In [8]:
len(latitudes)

35

In [9]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Location 0 successful
Location 1 successful
Location 2 successful
Location 3 successful
Location 4 successful
Location 5 successful
Location 6 successful
Location 7 successful
Location 8 successful
Location 9 successful
Location 10 successful
Location 11 successful
Location 12 successful
Location 13 successful
Location 14 successful
Location 15 successful
Location 16 successful
Location 17 successful
Location 18 successful
Location 19 successful
Location 20 successful
Location 21 successful
Location 22 successful
Location 23 successful
Location 24 successful
Location 25 successful
Location 26 successful
Location 27 successful
Location 28 successful
Location 29 successful
Location 30 successful
Location 31 successful
Location 32 successful
Location 33 successful
index 34 is out of bounds for axis 0 with size 34
Location 34 failed


(19.940137446240488, 0.45472188092156024)

In [10]:
rmse_list

[18.34175620077039,
 14.89516816727961,
 9.080491859092863,
 17.481187936568688,
 20.96525506086254,
 24.44449108335595,
 24.89814942093462,
 10.379296271569423,
 35.760902835373805,
 19.945454272388435,
 24.32439242598901,
 21.48797351535103,
 33.52032472927006,
 34.107523077573795,
 10.576123162332461,
 19.423491982676957,
 23.81842187875786,
 16.818692153477254,
 18.811074249755922,
 10.932778990584968,
 40.30247534267074,
 8.799535616735671,
 16.934761163975008,
 22.241168087211445,
 22.642804451008583,
 16.89452033508089,
 19.92300494118627,
 19.440327016185094,
 16.706179157925494,
 11.15748804335201,
 9.825233113852722,
 19.885301898087032,
 10.991295357168884,
 32.2076293737712]

In [11]:
mape_list

[0.5408992353111689,
 0.666015066541813,
 0.32936736071111,
 0.5547086979159785,
 0.3945982385300032,
 0.3034921409008256,
 0.6256634620059499,
 0.19561724861818583,
 0.34117088320098377,
 0.40495966873751793,
 0.3235036441630674,
 0.7879564923232223,
 1.0047419672543083,
 0.3598397976406427,
 0.4177349177328254,
 0.46029656176640327,
 0.33881902137971104,
 0.25392099980935395,
 0.5881628459452052,
 0.3899889731229063,
 0.5344646719733501,
 0.31758605544196317,
 0.6928475224667948,
 0.7032426918277074,
 0.6554887699353212,
 0.34269275968676177,
 0.2735728022039775,
 0.316462627287529,
 0.4728441082405393,
 0.3256345970837768,
 0.21945219680665615,
 0.30210845974623113,
 0.28236750853209586,
 0.7403219564891591]